## 크롬 브라우저 실행

In [41]:
# coding = utf-8

from selenium import webdriver

browser = webdriver.Chrome('/Applications/chromedriver')

## get 함수 웹 페이지 접속

In [42]:
browser.get("http://www.seibro.or.kr/websquare/control.jsp?w2xPath=/IPORTAL/user/company/BIP_CNTS01042V.xml&menuNo=286#")

## 조회 버튼 클릭

In [43]:
# find_element_by_id() : 태그명 찾는 함수
search_btn = browser.find_element_by_id("image1")
search_btn.click()

## 표 선택 및 DataFrame 변환

In [44]:
# brower 객체의 page_source 값 호출, html 소스 얻음
html = browser.page_source

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

# html 소스 파싱
soup = BeautifulSoup(html, "html.parser")
# id : grid1_body_table의 table 태그 값 찾음
table = soup.find("table", attrs={"id":"grid1_body_table"})
# Pandas DataFrame으로 변환
p = parser.make2d(table)
df = pd.DataFrame(p[2:], columns=p[1])
df.head()

,순위,종목코드,종목명,주식종류,시장구분,주당배당금,시가배당율,액면가배당율,액면가,결산월
0,1,084670,동양고속,보통주,유가증권시장,"4,700",16.9,94.00,"5,000",12
1,2,034950,한국기업평가,보통주,코스닥시장,"8,618",14.3,172.36,"5,000",12
2,3,172580,하이골드오션12호국제선박투자회사,보통주,유가증권시장,0,13.1,7.32,"5,000",12
3,4,095720,웅진씽크빅,보통주,유가증권시장,310,12.6,62.00,500,12
4,5,155900,바다로19호선박투자회사,보통주,유가증권시장,0,11.8,7.00,"5,000",12


## 페이지 이동하며 table 데이터 수집

In [45]:
import time
import random
from tqdm import tqdm

prev_no = 0
prev_table = None

# tqdm : 진행바 표시
for i in tqdm(range(0,200)):
    # 예외처리, 버튼 찾기 2초 간격으로 3번 시도
    try:
        next_btn = browser.find_element_by_id("cntsPaging01_next_btn")
        next_btn.click()
    except:
        time.sleep(2)
        try:
            next_btn = browser.find_element_by_id("cntsPaging01_next_btn")
            next_btn.click()
        except:
            time.sleep(2)
            next_btn = browser.find_element_by_id("cntsPaging01_next_btn")
            next_btn.click()

    # 로딩 정보 확인(bold 번호 확인 : w2pageList_control_label w2pageList_label_selected, html table 태그 정보 변화 확인)
    # 바뀔 때까지 1초 간격 3번 시도
    def get_html(browser, cnt):

        if cnt>=4:
            return -1, -1

        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        cur_no = soup.find("a", attrs={"class": "w2pageList_control_label w2pageList_label_selected"})
        cur_no = cur_no.text

        table = soup.find("table", attrs={"id": "grid1_body_table"})

        if cur_no!=prev_table and prev_table!=table:
            return cur_no, table
        else:
            time.sleep(1)
            get_html(browser, cnt+1)

    cur_no, table = get_html(browser, 1)

    if cur_no == -1:
        print("/n종료. 테이블 정보가 바뀌지 않았습니다.")
        break
    p = parser.make2d(table)
    temp = pd.DataFrame(p[2:], columns=p[1])
    df = pd.concat([df, temp],0)
    prev_no = cur_no
    prev_table = html

    if cur_no == "96":
        print("\n최종 페이지 도달")
        break

    # IP 차단 방지를 위한 3~5초 휴식
    time.sleep(random.randrange(3,5))

df.to_excel("stock_div.xlsx")

 47%|████▋     | 94/200 [05:48<06:32,  3.71s/it]



최종 페이지 도달


In [48]:
data = pd.read_excel('stock_div.xlsx')
data

,Unnamed: 0,순위,종목코드,종목명,주식종류,시장구분,주당배당금,시가배당율,액면가배당율,액면가,결산월
0,0,1,084670,동양고속,보통주,유가증권시장,"4,700",16.9,94.00,"5,000",12
1,1,2,034950,한국기업평가,보통주,코스닥시장,"8,618",14.3,172.36,"5,000",12
2,2,3,172580,하이골드오션12호국제선박투자회사,보통주,유가증권시장,0,13.1,7.32,"5,000",12
3,3,4,095720,웅진씽크빅,보통주,유가증권시장,310,12.6,62.00,500,12
4,4,5,155900,바다로19호선박투자회사,보통주,유가증권시장,0,11.8,7.00,"5,000",12
...,...,...,...,...,...,...,...,...,...,...,...
1435,10,1421,123900,골든블루,보통주,프리보드시장,40,0.0,8.00,500,12
1436,11,1422,105915,케이티디에스1우,우선주,기타비상장,0,0.0,43.40,"5,000",12
1437,12,1423,139150,이지스자산운용,보통주,기타비상장,0,0.0,98.60,500,12
1438,13,1424,141110,에이플러스라이프,보통주,프리보드시장,0,0.0,0.50,500,12
